In [1]:
%reload_ext autoreload
%autoreload 2
from pathlib import Path
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")
from dataset import ManualFeatureDataset, ManualFeatureDataModule

## Train/Test Split

In [2]:
root_dir = Path("/media/nvme1/icare-data/6h-features")
labels_csv = Path("/home/bc299/icare/patient_data.csv")

## Dataset Setup

In [3]:
all_patient_ids = [dir_.name for dir_ in root_dir.iterdir()]
train_ids, temp_ids = train_test_split(all_patient_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=2/3, random_state=42)

In [4]:
dataset = ManualFeatureDataset(root_dir, labels_csv, train_ids)

In [5]:
data_module = ManualFeatureDataModule(root_dir=root_dir,
                                      labels_csv=labels_csv,
                                      batch_size=32)
data_module.setup()

## Train Model

In [12]:
from model import BiLSTMClassifierModule

In [13]:
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer

data_module = ManualFeatureDataModule(root_dir, labels_csv, batch_size=32)
model = BiLSTMClassifierModule(input_size=8, hidden_size=128, num_layers=4, dropout=0.5, learning_rate=1e-5)
logger = WandbLogger(project="test-project")
trainer = Trainer(max_epochs=10, logger=logger)

/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:398: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(model, data_module)

/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory ./test-project/so6r6gvd/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type             | Params
------------------------------------------------
0  | model     | BiLSTMClassifier | 1.3 M 
1  | train_acc | BinaryAccuracy   | 0     
2  | val_acc   | BinaryAccuracy   | 0     
3  | test_acc  | BinaryAccuracy   | 0     
4  | train_auc | BinaryAUROC      | 0     
5  | val_auc   | BinaryAUROC      | 0     
6  | test_auc  | BinaryAUROC      | 0     
7  | train_f1  | BinaryF1Score    | 0     
8  | val_f1    | BinaryF1Score    | 0     
9  | test_f1   | BinaryF1Score    | 0     
10 | test_roc  | BinaryROC        | 0     
------------------------------------------------
1.3 M     Trainable params
0         Non-trainable para

/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1:  13%|█▎        | 16/123 [00:00<00:00, 128.80it/s, v_num=6gvd, train_loss_step=0.679, train_acc_step=0.719, train_auc=0.500, train_f1=0.836, val_loss=0.687, val_acc=0.627, val_auc=0.500, val_f1=0.771, train_loss_epoch=0.693, train_acc_epoch=0.514] 

/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 7:  28%|██▊       | 34/123 [00:00<00:00, 129.56it/s, v_num=6gvd, train_loss_step=0.610, train_acc_step=0.781, train_auc=0.500, train_f1=0.877, val_loss=0.662, val_acc=0.627, val_auc=0.500, val_f1=0.771, train_loss_epoch=0.676, train_acc_epoch=0.591] 

In [10]:
trainer.test(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 11/11 [00:00<00:00, 252.85it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test_acc_12        │    0.5980392156862745     │
│        test_acc_18        │    0.5980392156862745     │
│        test_acc_24        │    0.5980392156862745     │
│        test_acc_30        │    0.5980392156862745     │
│        test_acc_36        │    0.5980392156862745     │
│        test_acc_42        │    0.5980392156862745     │
│        test_acc_48        │    0.5980392156862745     │
│        test_acc_54        │    0.5980392156862745     │
│        test_acc_60        │    0.5980392156862745     │
│        test_acc_66        │    0.5980392156862745     │
│        test_acc_72        │    0.5980392156862745     │
│        test_auc_12        │            0.5            │
│        test_auc_18        │            0.5            │
│        test_auc_24        │            0.5            │
│        test_auc_30        │            0.5            │
│        test_auc_36        │            0.5            │
│        test_auc_42        │            0.5            │
│        test_auc_48        │            0.5            │
│        test_auc_54        │            0.5            │
│        test_auc_60        │            0.5            │
│        test_auc_66        │            0.5            │
│        test_auc_72        │            0.5            │
│        test_f1_12         │    0.7484662532806396     │
│        test_f1_18         │    0.7484662532806396     │
│        test_f1_24         │    0.7484662532806396     │
│        test_f1_30         │    0.7484662532806396     │
│        test_f1_36         │    0.7484662532806396     │
│        test_f1_42         │    0.7484662532806396     │
│        test_f1_48         │    0.7484662532806396     │
│        test_f1_54         │    0.7484662532806396     │
│        test_f1_60         │    0.7484662532806396     │
│        test_f1_66         │    0.7484662532806396     │
│        test_f1_72         │    0.7484662532806396     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc_12': 0.5980392156862745,
  'test_auc_12': 0.5,
  'test_f1_12': 0.7484662532806396,
  'test_acc_18': 0.5980392156862745,
  'test_auc_18': 0.5,
  'test_f1_18': 0.7484662532806396,
  'test_acc_24': 0.5980392156862745,
  'test_auc_24': 0.5,
  'test_f1_24': 0.7484662532806396,
  'test_acc_30': 0.5980392156862745,
  'test_auc_30': 0.5,
  'test_f1_30': 0.7484662532806396,
  'test_acc_36': 0.5980392156862745,
  'test_auc_36': 0.5,
  'test_f1_36': 0.7484662532806396,
  'test_acc_42': 0.5980392156862745,
  'test_auc_42': 0.5,
  'test_f1_42': 0.7484662532806396,
  'test_acc_48': 0.5980392156862745,
  'test_auc_48': 0.5,
  'test_f1_48': 0.7484662532806396,
  'test_acc_54': 0.5980392156862745,
  'test_auc_54': 0.5,
  'test_f1_54': 0.7484662532806396,
  'test_acc_60': 0.5980392156862745,
  'test_auc_60': 0.5,
  'test_f1_60': 0.7484662532806396,
  'test_acc_66': 0.5980392156862745,
  'test_auc_66': 0.5,
  'test_f1_66': 0.7484662532806396,
  'test_acc_72': 0.5980392156862745,
  'test_auc_7